# 掛載

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


read

In [ ]:
pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.0 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import numpy as np

# 图像文件夹路径
folder_0 = '/content/drive/MyDrive/Colab_Notebooks/data_1/0/'
folder_1 = '/content/drive/MyDrive/Colab_Notebooks/data_1/1/'

# 读取并处理图像
x_data = []

# 处理文件夹0中的图像
for filename in os.listdir(folder_0):
    img = cv2.imread(os.path.join(folder_0, filename))
    img = cv2.resize(img, (64, 64))  # 调整图像大小为模型输入大小
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转换为灰度图像
    img = img / 255.0  # 归一化像素值
    x_data.append(img)

# 处理文件夹1中的图像
for filename in os.listdir(folder_1):
    img = cv2.imread(os.path.join(folder_1, filename))
    img = cv2.resize(img, (64, 64))  # 调整图像大小为模型输入大小
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # 转换为灰度图像
    img = img / 255.0  # 归一化像素值
    x_data.append(img)

# 将图像数据转换为NumPy数组
x_data = np.array(x_data)

# 输出 x_data 的形状
print("x_data shape:", x_data.shape)


x_data shape: (150, 64, 64)


CNN

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Class2(nn.Module):
    def __init__(self):
        super(Class2, self).__init__()

        # 卷積層
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, \
                               stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, \
                               stride=1, padding=1)

        # 線性層
        self.linear1 = nn.Linear(in_features=65536, out_features=256)
        self.linear2 = nn.Linear(in_features=256, out_features=64)

        # 隨機失活(Dropout)
        self.dropout = nn.Dropout(p=0.25)

        # 最後一個線性層輸出2個值，代表2個類別的機率
        self.linear3 = nn.Linear(in_features=64, out_features=2)

    def forward(self, image_batch):
        image_batch = F.relu(input=self.conv1(image_batch))
        image_batch = F.relu(input=self.conv2(image_batch))

        flat_image_batch = image_batch.view(image_batch.shape[0], -1)

        flat_image_batch = F.relu(input=self.linear1(flat_image_batch))
        flat_image_batch = self.dropout(F.relu(input=self.linear2(flat_image_batch)))
        probabilities = F.softmax(self.linear3(flat_image_batch), dim=1)  # 使用softmax轉換成機率

        return probabilities


In [ ]:
!pip install torchsummary

In [ ]:
np.dtype(np.int32)

dtype('int32')

In [ ]:
import torch.optim as optim
import torch
import torch.nn as nn
criterion = nn.CrossEntropyLoss()  # 定義損失函數
# 使用 torch.optim 設置優化器函數作為優化庫
learning_rate = 0.1

data= torch.from_numpy(x_data)
model = Class2()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
model

Class2(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (linear1): Linear(in_features=65536, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=64, bias=True)
  (dropout): Dropout(p=0.25, inplace=False)
  (linear3): Linear(in_features=64, out_features=2, bias=True)
)

In [ ]:
data=data.to(torch.float)

In [ ]:
data = data.unsqueeze(1)  # 在第二个维度上添加通道维度

In [ ]:
data.shape

torch.Size([150, 1, 64, 64])

In [ ]:
model(data)

tensor([[0.5080, 0.4920],
        [0.5092, 0.4908],
        [0.5106, 0.4894],
        [0.5090, 0.4910],
        [0.5114, 0.4886],
        [0.5049, 0.4951],
        [0.5126, 0.4874],
        [0.5116, 0.4884],
        [0.5102, 0.4898],
        [0.5083, 0.4917],
        [0.5094, 0.4906],
        [0.5114, 0.4886],
        [0.5080, 0.4920],
        [0.5109, 0.4891],
        [0.5075, 0.4925],
        [0.5063, 0.4937],
        [0.5074, 0.4926],
        [0.5043, 0.4957],
        [0.5097, 0.4903],
        [0.5131, 0.4869],
        [0.5106, 0.4894],
        [0.5116, 0.4884],
        [0.5144, 0.4856],
        [0.5186, 0.4814],
        [0.5135, 0.4865],
        [0.5085, 0.4915],
        [0.5096, 0.4904],
        [0.5018, 0.4982],
        [0.5108, 0.4892],
        [0.5122, 0.4878],
        [0.5070, 0.4930],
        [0.5074, 0.4926],
        [0.5061, 0.4939],
        [0.5122, 0.4878],
        [0.5088, 0.4912],
        [0.5134, 0.4866],
        [0.5010, 0.4990],
        [0.5141, 0.4859],
        [0.5

In [ ]:


def inference():
    PATH = 'model_CNN.pth' # 保存模型的路径
    # 加载模型
    model = Class2().to('cpu') # 实例化我们的PyTorch模型
    model.load_state_dict(torch.load(PATH)) # 加载保存的模型
    # summary(model, (1, 28, 28), device='cpu') # 检查是否一切都正确

    # 获取一些'test_loader'数据用于我们的模型
    dataiter = iter(test_loader) # 获取整个批次
    images, labels = next(dataiter) # 提取图像和它们的标签
    label = labels[0] # 真实标签

    # 图像需要一些处理！
    image = images[0] # 从批次中取第一张图像 = 1 x 28 x 28
    # 记住，我们的模型期望第一个维度作为'批大小'
    # 因此，在'开始'添加一个额外的维度。
    image = torch.unsqueeze(image, dim=0) # 1 x 1 x 28 x 28 (批大小 = 1)
    print('\nimage.shape =>', image.shape)

    # 禁用自动微分模块
    with torch.no_grad(): # 不需要计算梯度，因为我们不在训练
        # 进行前向传播
        output = model.forward(image)

    # 我们的模型输出'logits'，我们需要将其转换为类概率。
    # https://discuss.pytorch.org/t/how-to-extract-probabilities/2720/12
    # 要转换logits，我们需要使用 'Softmax' 函数
    # https://medium.com/data-science-bootcamp/understand-the-softmax-function-in-minutes-f3a59641e86d
    # 因此，
    class_probabilities = F.softmax(output, dim=1).numpy().squeeze()
    print('\n类别概率 ==>', class_probabilities)
    for i, proba in enumerate(class_probabilities):
        print(f'类别 \t{i}\t 概率 \t{100*proba:.2f}%')

    # 一个非常漂亮的方式来展示结果
    # 创建一个具有两个坐标轴的图，ax1和ax2
    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2) # 有2列的子图
    # ax1 显示来自测试数据集的图像
    ax1.imshow(image.resize_(1, 28, 28).numpy().squeeze())
    ax1.set_title('实际标签 ' + str(label.numpy()))
    # ax2 包含包含类别概率的水平条形图
    ax2.barh(np.arange(10), class_probabilities)
    ax2.set_aspect(0.1) # 调整ax2的纵横比，否则它会变得太大
    ax2.set_yticks(np.arange(10)) # 10个类别的y轴上有10个刻度
    ax2.set_yticklabels(np.arange(10)) # 设置刻度标签从0到9
    ax2.set_title('类别概率')
    ax2.set_xlim(0, 1.1) # 概率不能超过1，因此将限制设置为1.1

    return None
